In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_mistralai import ChatMistralAI
model = ChatMistralAI(model="mistral-large-latest")

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

In [10]:
class Classify(BaseModel):
    direction: str = Field(description="The predicted direction of the stock price movement", enum=["up", "down"])
    slope: int = Field(description="The predicted radicality of the stock price movement on the scale from 1 to 10, where 1 is 'random flunctuation' and 10 is 'certain, almost vertical slope'")

template = ChatPromptTemplate.from_template("""
Your task is to predict some properties of a time series given you by the user. 

Only extract the properties mentioned in the 'Classification' function.

Time series:
{inp}
""")

structured_llm = model.with_structured_output(Classify)
prompt = template.invoke({"inp": "1 100 98 200 198 300"})

response = structured_llm.invoke(prompt)
response

Classify(direction='up', slope=9)

In [11]:
from langchain_community.document_loaders import PyPDFLoader

In [12]:
loader = PyPDFLoader('/home/herb/Documents/private/MLE_CV.pdf')
doc = loader.load()[0]
doc.page_content

'Matvey Kamenev\n+7 (922) 244-99-77 | mvkamenev@edu.hse.ru | t.me/rumatstone | github.com/herbitovich\nEducation\nHigher School of Economics Saint-Petersburg, Russia\nBachelor of Applied Data Analysis and Artificial Intelligence Sep. 2024 – June 2028\nHigher School of Economics Lyceum Perm, Russia\nMathematics and computer science advanced studies class Sep. 2022 – June 2024\nExperience\nLLM Researcher Apr. 2024 – Present\nPISH (VK&HSE Project Lab) Saint-Petersburg, Russia\n• Developed a cross-architecture adapter module, which allowed HuggingFace models integration into the\nexperimentation framework\n• Conducted and interpreted 30+ architecture experiments on multiple tasks and benchmarks to evaluate\ndifferent models’ performance\n• Explored various methods of linear-time attention architectures\n• Read, analyzed and proposed new architecture-improving ideas based on 10+ NLP arxiv papers\nNLP Machine Learning Engineer Sep. 2024 – Feb. 2025\nNDA outsourcing Saint-Petersburg, Russia\n

In [16]:
from typing import List, Optional

class Occupation(BaseModel):
    """Information about one of the applicant's places of employment."""
    company_name: Optional[str] = Field(description="Name of the company. Output null if the provided information is not a real legal entity, like 'Personal Project'.")
    position_name: str = Field(description="Name of the position that the applicant has/had in the company.")
    duration: int = Field(description="Number of months that the applicant has/had been working in the company.")
class Person(BaseModel):
    """Information about the applicant and their working experience"""
    name: Optional[str] = Field(desciption="Full name of the applicant. Output null if not provided.")
    employment_history: List[Occupation] = Field(desciption="The whole employment history of the applicant.")
    total_experience: int = Field(desciption="Total working experience duration in months. Output 0 if not provided.")
screener = model.with_structured_output(Person)
template = ChatPromptTemplate.from_messages(
    [("system", "You are to extract the information about a job applicant. Follow the provided schema."),
     ("user", "Here is the job application that I want you to scrape for the relevant info:\n{application}")]
)

prompt = template.invoke({"application": doc.page_content})
res = screener.invoke(prompt)
res.model_dump()

{'name': 'Matvey Kamenev',
 'employment_history': [{'company_name': 'PISH (VK&HSE Project Lab)',
   'position_name': 'LLM Researcher',
   'duration': 5},
  {'company_name': 'NDA outsourcing',
   'position_name': 'NLP Machine Learning Engineer',
   'duration': 5},
  {'company_name': None,
   'position_name': 'Python Backend Developer',
   'duration': 17}],
 'total_experience': 27}